<a href="https://colab.research.google.com/github/huantp/DataCamp/blob/master/Weebly_Google_Play_ASO_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraps Google Play app's info 



1.   Scraps Google Play's apps's info using keywwords
2.   Sorts apps's according to the number of reviews,etc.



In [ ]:
#Mount's Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Function: Installs the API that scraps GooglePlay
!pip install google-play-scraper

     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-0.1.2-cp36-none-any.whl size=22280 sha256=9b6fd12ce03a8771a31abd38a64cd78bf1fbbfef49d4a084365d31000dc967d4
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dc/847f7da787b7d2c28b26a5062f86eac3f16a8cb0bdfdc1f308
Successfully built google-play-scraper


In [7]:
#Function: Gets GP data for list of countries, languages, keywords


#Import modules 
from bs4 import BeautifulSoup
from google_play_scraper import app
import pandas as pd
import requests
import re
from google.colab import files
from datetime import date
from collections import OrderedDict




def get_gp(keyword_1,keyword_2):
  '''gets data from google play based on keywords and 
  three countries (Germany, France, Italy')'''

  gp_words = [keyword_1,keyword_2]
  languages = ['de' ,'fr','it'] #We will add all the countries
  countries = ['de' ,'fr','it']

  #Lists created by looping over variables and extraction app
  app_list = [] #Addresses of each app
  df_list = []  #List of dataframes. Each dataframe has its own keyword, language and country
  #Extracting data
  for kword in gp_words:
    for kcoun in countries:
      r = requests.get(f'https://play.google.com/store/search?q={kword}&c=apps&hl={kcoun}&gl={kcoun}') #Not use country for both language and country to avoid overcomplicate it
      soup = BeautifulSoup(r.text) 
      txt=soup.prettify()
      pattern = "(com..*?)\,7"  #Gets all the adresses coming bedore /7 and starting with com. Tocheck: is it 100% precise?
      addresses=re.findall(pattern,txt)
      addresses = list(OrderedDict.fromkeys(addresses)) #Delete duplicates and keep order
      #We apply the API app and choose country and language
      for address in addresses:
        try:
          result = app(address,lang= kcoun, country= kcoun) #We set the same language and country to reduce amount of data
          app_list.append(result)
          df= pd.DataFrame(app_list)
          df['keyword'] = kword
          #df['language'] = language Don't add to avoid over complicate data
          df['country'] = kcoun
          #df['rank'] = enumerate(address) #To_do: check if they can be ranked by popularity.
          df['info_extracted'] =pd.to_datetime("today") #To_do: If True not fill it up (so doesn't superseed the former date)
          df_list.append(df)
        except :   #This why we might/or not lose some data
          continue
        else:
         continue

  df = pd.concat(df_list) #We concatenate the list of dataframes so we can have one datafra
  df = df.drop_duplicates(subset='title', keep="first") #We are dropping thousands of duplicates. 
  
  return df

#Gets data and display first two rows
df = get_gp('Poker','Casino') #Function that scrapes Google Play
df.head(2)

,title,description,descriptionHTML,summary,summaryHTML,installs,minInstalls,score,ratings,reviews,histogram,price,free,currency,sale,saleTime,originalPrice,saleText,offersIAP,inAppProductPrice,size,androidVersion,androidVersionText,developer,developerId,developerEmail,developerWebsite,developerAddress,privacyPolicy,developerInternalID,genre,genreId,icon,headerImage,screenshots,video,videoImage,contentRating,contentRatingDescription,adSupported,containsAds,released,updated,version,recentChanges,recentChangesHTML,comments,editorsChoice,appId,url,keyword,country,info_extracted
0,Zynga Poker: Texas Holdem - Online Casino Spie...,Mach mit bei einem der beliebtesten kostenlose...,Mach mit bei einem der beliebtesten kostenlose...,Gewinne groß und besiege deine Freunde! Spiele...,Gewinne groß und besiege deine Freunde! Spiele...,50.000.000+,50000000,4.293116,2550733.0,738401,"[284221, 49432, 109222, 299446, 1808412]",0,True,USD,False,None,None,None,True,"0,86 € bis 344,00 € pro Artikel",97M,4.4,4.4 oder höher,Zynga,6690081412016968981,android.support@zynga.com,https://www.zynga.com/,"699 8th Street, San Francisco. CA",https://www.zynga.com/privacy/policy,6690081412016968981,Casino,GAME_CASINO,https://play-lh.googleusercontent.com/JYfNHIlJ...,https://play-lh.googleusercontent.com/Ixm355D7...,[https://play-lh.googleusercontent.com/A5g-2cU...,None,None,USK ab 16 Jahren,None,True,True,21.12.2010,1613079013,22.07,Das ist neu in Zynga Poker:\r\n– NEUE Winterwu...,Das ist neu in Zynga Poker:<br>– NEUE Winterwu...,[],False,com.zynga.livepoker,https://play.google.com/store/apps/details?id=...,Poker,de,2021-02-12 19:01:41.507604
1,Governor of Poker 3 : Texas Holdem mit Freunden,<b>Bekannt aus dem TV!</b>\r\nSpiele Online-Po...,<b>Bekannt aus dem TV!</b><br>Spiele Online-Po...,Kostenloses Kartenspiel Multiplayer-Texas-Hold...,Kostenloses Kartenspiel Multiplayer-Texas-Hold...,5.000.000+,5000000,4.278679,253761.0,71339,"[29270, 6023, 10212, 27465, 180791]",0,True,USD,False,None,None,None,True,"0,99 € bis 199,99 € pro Artikel",124M,4.4,4.4 oder höher,Youda Games Holding B.V.,7639378078513286726,support@governorofpoker.com,http://www.governorofpoker.com,Beechavenue 182\n1119 PX Schiphol-Rijk\nThe Ne...,http://www.governorofpoker.com/privacy/,7639378078513286726,Kartenspiele,GAME_CARD,https://play-lh.googleusercontent.com/rq6H2UQ1...,https://play-lh.googleusercontent.com/C-STOMc3...,[https://play-lh.googleusercontent.com/nrd0sI1...,https://www.youtube.com/embed/NOAJafeNLfE?ps=p...,https://i.ytimg.com/vi/NOAJafeNLfE/hqdefault.jpg,USK ab 16 Jahren,None,True,True,29.09.2014,1612456543,7.6.0,This Governor of Poker 3 update brings:\r\n- P...,This Governor of Poker 3 update brings:<br>- P...,[],False,com.youdagames.gop3multiplayer,https://play.google.com/store/apps/details?id=...,Poker,de,2021-02-12 19:01:41.687465


In [ ]:
#We store raw data on csv in GoogleDrive
import glob
from google.colab import files


#Converts file to csv and names it according data
today = pd.to_datetime("today").strftime('%Y-%m-%d')
file_title = f'df_gp_raw_{today}.csv'
df.to_csv(file_title)




In [9]:
#Stores the csv on your google drive
#Todo: this file-name still needs to be written manually,change to automatic
#Note: needs to write your own pathfile after .csv'

!cp 'df_gp_raw_2021-02-12.csv' '/content/drive/MyDrive/1  Python & Data Science /Weebly Python'  #To_do: avoid writing the finle name as string


#Updates data, cleans it and produce final dataframe


1.   Loads old csv
2.   Concatenates it with new csv data
1.   Chooses columns/info we want to analyse in detail
2.   Cleans up info/format
1.   Exports it to csv



In [10]:
#Gets list of similar files in our Google Drive
data_files = glob.glob("/content/drive/MyDrive/1  Python & Data Science /Weebly Python/df_gp_raw*")
file_oldest = data_files[0] #We get the df stored
#Note: we subset the csv to get key = title, reviews, ratings, score for future analysis

data_files

['/content/drive/MyDrive/1  Python & Data Science /Weebly Python/df_gp_raw_2021-02-02.csv',
 '/content/drive/MyDrive/1  Python & Data Science /Weebly Python/df_gp_raw_2021-02-12.csv']

In [14]:
#Cleans up data

#We get new data 
df_old = pd.read_csv(file_oldest) #Drive is already mounted #Check formats (datetime) are correct
df_new = df.copy() 

#Dates: 
  #Make sure info-extracted is properly formated to date time, fill nan values
df_old['info_extracted_date'] = df_old['info_extracted'].fillna(method='ffill')
df_new['info_extracted_date'] = df_new['info_extracted'].fillna(method='ffill')

df_old['info_extracted_date']= pd.to_datetime(df_old['info_extracted_date'], errors = 'coerce')
df_new['info_extracted_date']= pd.to_datetime(df_new['info_extracted_date'], errors = 'coerce')


#Column installs. Delete string and convert to number
df_old["installs_n"] = df_old['installs'].str.replace(r'\D', '')
df_new["installs_n"] = df_old['installs'].str.replace(r'\D', '')

df_old["installs_n"] = pd.to_numeric(df_old["installs_n"])
df_new["installs_n"] = pd.to_numeric(df_new["installs_n"])


#Column size. Deletes string an dconvert to number
df_old['size_n'] = df_old['size'].str.replace(r'\D', '')
df_new['size_n'] = df_new['size'].str.replace(r'\D', '')

#We drop nan values for these columns
df_old.dropna(subset=['title', 'ratings','reviews'],how ='any')
df_new.dropna(subset=['title', 'ratings','reviews'],how ='any')

print('(important) data cleaned up')


(important) data cleaned up


# Analyses growth in ratings, reviews

In [15]:
import numpy as np


#Select key_growth: columns needed to analyse growth of the app

#To_do:optimize selection of columns
key_growth =['title','country','installs_n','score','ratings',
          'reviews','size_n','developer',
          'released','info_extracted_date']

keys_gr_merge =['title','country','installs_n',
           'size_n','developer','released'] 

#Subset raw data to analyse it for growth
df_n = df_new[key_growth] 
df_o =df_old[key_growth]



#merge both dataframes
df_merged = pd.merge(df_n, df_o, how="left", on = keys_gr_merge )


#Get the differences of reviews, ratings
df_merged['reviews_diff'] = df_merged['reviews_x']-df_merged['reviews_y']
df_merged['ratings_diff'] = df_merged['ratings_x']-df_merged['ratings_y']
oneday = pd.Timedelta(days=1)
#df_merged['diff_days'] = (df_merged['info_extracted_x']-result['info_extracted_y'])/np.timedelta64(1, 'D') #Sure this is days?
df_merged['diff_days'] = (df_merged['info_extracted_date_x']-df_merged['info_extracted_date_y']).dt.days

#NOTE: REVIEW THIS 2020/2/4
if df_merged['diff_days'].any()<1:
  df_merged['diff_days'] = 1
else:
  pass

df_merged['reviews_by_day'] =df_merged['reviews_diff']/df_merged['diff_days']
df_merged['ratings_by_day'] =df_merged['ratings_diff']/df_merged['diff_days']

#Selects the columns we are interested to analyse growth by reviews and ratings
df_growth = df_merged[['title','country','installs_n','size_n','developer',
                   'released','reviews_by_day','ratings_by_day']]

df_growth.head(2)


,title,country,installs_n,size_n,developer,released,reviews_by_day,ratings_by_day
0,Zynga Poker: Texas Holdem - Online Casino Spie...,de,50000000.0,97,Zynga,21.12.2010,134.4,372.1
1,Governor of Poker 3 : Texas Holdem mit Freunden,de,1000000.0,124,Youda Games Holding B.V.,29.09.2014,NaN,NaN


In [17]:
#Analyses growth in terms of number of reviews and ratings

import numpy as np

#merge both dataframes
df_merged = pd.merge(df_n, df_o, how="left", on = keys_gr_merge )


#Get the differences of reviews, ratings
df_merged['reviews_diff'] = df_merged['reviews_x']-df_merged['reviews_y']
df_merged['ratings_diff'] = df_merged['ratings_x']-df_merged['ratings_y']
oneday = pd.Timedelta(days=1)
#df_merged['diff_days'] = (df_merged['info_extracted_x']-result['info_extracted_y'])/np.timedelta64(1, 'D') #Sure this is days?
df_merged['diff_days'] = (df_merged['info_extracted_date_x']-df_merged['info_extracted_date_y']).dt.days

#NOTE: REVIEW THIS 2020/2/4
if df_merged['diff_days'].any()<1:
  df_merged['diff_days'] = 1
else:
  pass

df_merged['reviews_by_day'] =df_merged['reviews_diff']/df_merged['diff_days']
df_merged['ratings_by_day'] =df_merged['ratings_diff']/df_merged['diff_days']


df_growth = df_merged[['title','country','installs_n','size_n','developer',
                   'released','reviews_by_day','ratings_by_day']]

df_growth.head(2)

,title,country,installs_n,size_n,developer,released,reviews_by_day,ratings_by_day
0,Zynga Poker: Texas Holdem - Online Casino Spie...,de,50000000.0,97,Zynga,21.12.2010,134.4,372.1
1,Governor of Poker 3 : Texas Holdem mit Freunden,de,1000000.0,124,Youda Games Holding B.V.,29.09.2014,NaN,NaN


In [18]:
#KPIs: df_growthreviews_by_day and ratings_by_day 

df_growth.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177 entries, 0 to 176
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           177 non-null    object 
 1   country         177 non-null    object 
 2   installs_n      133 non-null    float64
 3   size_n          177 non-null    object 
 4   developer       177 non-null    object 
 5   released        177 non-null    object 
 6   reviews_by_day  26 non-null     float64
 7   ratings_by_day  26 non-null     float64
dtypes: float64(3), object(5)
memory usage: 12.4+ KB


#Next: Connect to DataStudio